# **Infer `Cellmask` and `Nucleus`**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb).***

### ➡️ **Input:**
In this workflow, a single or multi-channel confocal microscopy image of fluorescently tagged organelles will be used to "infer" (or segment) the cell and nucleus masks. The following setup is recommended for this pipeline:

| **Imaging Requirements**|[**Masks Workflow (A)**](./1.1a_infer_masks_from-composite_single_cell.ipynb)|
| :------------------------------------- |  :------:  |
| Nuclei Marker                          |      ✘     |
| Cell Membrane Marker                   |      ✘     |
| Cytoplasmic Organelles                 |      ✔     |
| Number of cells per image              |   Single   |

*If you images are not compatible with this setup, consider the other 1.1_Masks Workflows available:*

| **Imaging Requirements**|[**Masks Workflow**](./1.1_infer_masks_from-composite_with_nuc.ipynb)|[**Masks Workflow (B)**](./1.1b_infer_masks_from-composite_multiple-cells.ipynb)| 
| :------------------------------------- |  :------:  |  :------:  |
| Nuclei Marker                          |     ✔     |     ✘     |
| Cell Membrane Marker                   |     ✘     |     ✘     |
| Cytoplasmic Organelles                 |     ✔     |     ✔     |
| Number of cells per image              |  Single or Multiple  |  Multiple |

### **Included in this Notebook:**
1. **Infer *cytoplasm*** - Segment the ***cytoplasm*** from a composite image of multiple organelle markers combined. The ***cell*** and ***nucleus*** masks will be derived from this mask.
    
    *The combination of organelle markers used to create the composite image for the cytoplasm segmentation is based on the assumption that the organelle labels used will "fill up" the entire cytoplasm (not including the nucleus). This is NOT the most accurate method to determine the cell area, but is required in the case where membrane and nuclei markers can not be used or are not included. It is important to consider specifics of your system as the cell type and labeling method may differ from the example above.*

2. **Infer *nucleus*** - Segment the ***nucleus*** from an exclusive or (XOR) of the ***cytoplasm*** mask and a hole-filled version of the ***cytoplasm*** mask.
3. **Infer *cellmask***
Segment the ***cellmask*** by combining the ***nucleus*** and ***cytoplasm*** masks.
-----

### 👣 **Summary of steps**  

➡️ **EXTRACTION**
- **`STEP 1`** - Create composite image

    - determine weight to apply to each channel of the intensity image (w# = user input)
    - rescale summed image intensities (rescale = user input)

**PRE-PROCESSING**
- **`STEP 2`** - Rescale and smooth image

    - rescale intensity of composite image (min=0, max=1)
    - median filter (median size = user input)
    - gaussian filter (sigma = user input)
    
- **`STEP 3`** - Log transform + Scharr edge detection

    - perform a log transformation on the image
    - apply Scharr edge detection filter
    - combine log image and Scharr edge filtered intensity image
 
**CORE PROCESSING**
- **`STEP 4`** - Global + local thresholding (AICSSeg – MO)

    - apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)

**POST-PROCESSING**
- **`STEP 5`** - Remove small holes and objects

    - fill holes (hole size = user input)
    - remove small objects (object size = user input)
    - filter method = (method = user input)

**POST-POST-PROCESSING**
- **`STEP 6`** - Create nucleus mask from cytoplasm mask

    - binary dilation
    - fill nucleus (hole size = user input)
    - binary erosion
    - logical **XOR** of the cytoplasm and the filled in cytoplasm resulting in the nucleus and any artifacts from dilation/erosion
    - remove small objects (object size = user input)


- **`STEP 7`** - Create cellmask from cytoplasm and nucleus mask

    - logical **OR** of the nucleus and cytoplasm
    - fill small holes (hole size = user input)
    - remove small objects (object size = user input)

**OUTPUT** ➡️ 
- **`STEP 8`** - Stack masks

    - Stack masks in order of nucleus, cellmask and cytoplasm mask

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [4]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from skimage.morphology import binary_dilation, binary_erosion

from infer_subc.core.file_io import (read_czi_image, 
                                     export_inferred_organelle,
                                     list_image_files)
                  
from infer_subc.core.img import *
from infer_subc.organelles import non_linear_cellmask_transform

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `im_type`, `data_root_path`, `in_data_path`, and `out_data_path`.

In [5]:
#### USER INPUT REQUIRED ###
im_type = ".czi"
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
in_data_path = data_root_path / "raw"
out_data_path = data_root_path / "out"

#### &#x1F3C3; **Run code; no user input required**

In [6]:
# list files in the input folder
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

,Image Name
0,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\11252023_MSi08L_iPSCs_undiff_BR1_N20_Unmixing.czi
1,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\24hrs-Ctrl +oleicAcid 50uM_2_Unmixing.czi
2,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\24hrs-Ctrl +oleicAcid 50uM_3_Unmixing.czi
3,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a24hrs-Ctrl +oleicAcid 50uM_10_Unmixing.czi
4,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a24hrs-Ctrl +oleicAcid 50uM_4_Unmixing.czi
5,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a24hrs-Ctrl_10_Unmixing.czi
6,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a24hrs-Ctrl_14_Unmixing.czi
7,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a48hrs-Ctrl + oleic acid_01_Unmixing.czi
8,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a48hrs-Ctrl + oleic acid_02_Unmixing.czi
9,C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a48hrs-Ctrl + oleic acid_07_Unmixing.czi


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [7]:
#### USER INPUT REQUIRED ###
test_img_n = 5

#### &#x1F3C3; **Run code; no user input required**

In [8]:
# load image and metadata
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# metadata
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']
print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

# open viewer and add images
viewer = napari.Viewer()
for i in list(range(len(channel_names))):
    viewer.add_image(img_data[i],
                     scale=scale,
                     name=f"Channel {i}")
viewer.grid.enabled = True
viewer.reset_view()
print("\nProceed to Napari window to view your selected image.")

Metadata information
File path: C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\raw\a24hrs-Ctrl_10_Unmixing.czi
Channel 0 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Nuclei_Jan22
Channel 1 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Lyso+405_Jan22
Channel 2 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Mito+405_Jan22
Channel 3 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Golgi+405_Jan22
Channel 4 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Peroxy+405_Jan22
Channel 5 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: ER+405_Jan22
Channel 6 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: BODIPY+405low_Jan22
Channel 7 name: 0 :: a24hrs-Ctrl_10_Unmixing-0 :: Residuals
Scale (ZYX): (0.3891184878080979, 0.07987165184837317, 0.07987165184837318)
Channel axis: 0


20-Nov-24 13:28:39 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1924x1978+640-803 (frame: 1942x2025+631-841) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 1924x2255+641-796 (frame: 1942x2302+632-834) margins: 9, 38, 9, 9 minimum size: 385x515 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=403,562 maxtrack=0,0)



Proceed to Napari window to view your selected image.


-----

## **EXTRACTION**

### **`STEP 1` - Create composite image**

&#x1F453; **FYI:** This code block creates a composite image of the organelle channels. The intensity of each channel in combined together with specified weights. The resulting composite will be used to segment the cell area. *Hint: combine organelles in a way that produces a relatively even intensity throughout the cell.* 

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `w0`: the weight of channel 0 (the first channel); a value of 0 will exclude this channel from the compostite; large values will cause this channel to be more prominent in the final composite image
- `w1`: the weight of channel 1
- `w2`: the weight of channel 2
- `w3`: the weight of channel 3
- `w4`: the weight of channel 4
- `w5`: the weight of channel 5
- `w6`: the weight of channel 6
- `w7`: the weight of channel 7
- `w8`: the weight of channel 8
- `w9`: the weight of channel 9
- `rescale`: "True" rescales the image so that the pixel/voxel with the highest intensity is set to 1 and the one with the lowest intensity is set to 0; "False" leaves the combined intensity values as is after the composite is made

In [9]:
#### USER INPUT REQUIRED ###
w0 = 0
w1 = 4
w2 = 1
w3 = 1
w4 = 2
w5 = 2
w6 = 0
w7 = 0
w8 = 0
w9 = 0

rescale = True

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates the composite and optionally rescales the image based on the settings above. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the smoothing settings selected above.

In [10]:
# make aggregate and optionally rescale
struct_img_raw = make_aggregate(img_data,
               weight_ch0= w0,
               weight_ch1= w1,
               weight_ch2= w2,
               weight_ch3= w3,
               weight_ch4= w4,
               weight_ch5= w5,
               weight_ch6= w6,
               weight_ch7= w7,
               weight_ch8= w8,
               weight_ch9= w9,
               rescale = rescale)

# adding image to Napari as a new layer
viewer.grid.enabled = False
viewer.reset_view()
viewer.add_image(struct_img_raw, scale=scale, name="1-Cyto: Create composite")

<Image layer '1-Cyto: Create composite' at 0x12914c2b760>

-----
## **PRE-PROCESSING**

## **`STEP 2` - Rescale and smooth image**

&#x1F453; **FYI:** This code block rescales the image so that the pixel/voxel with the highest intensity is set to 1 and the one with the lowest intensity is set to 0. The image is then *optionally* smoothed using a Gaussian and/or median filter. 

<mark> Include more information on the Gaussian and median filtering methods here </mark>

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the amount of filter to use for each method. Higher values indicate more smoothing:
- `med_filter_size`: the size of the median filter to apply; if 0 is used, no filter will be applied
- `gaussian_smoothing_sigma`: the sigma to apply in the Gaussian filtering step; if 0 is used, no filter will be applied

In [11]:
#### USER INPUT REQUIRED ###
med_filter_size = 0
gaussian_smoothing_sigma = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block rescales the image and applies the specified median and Gaussian filters. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the smoothing settings selected above.

In [12]:
# rescaling and smoothing input image
structure_img_smooth = scale_and_smooth(struct_img_raw,
                                        median_size = med_filter_size, 
                                        gauss_sigma = gaussian_smoothing_sigma)


# adding image to Napari as a new layer
viewer.add_image(structure_img_smooth, scale=scale, name="2-Cyto: Rescale and Smooth")

<Image layer '2-Cyto: Rescale and Smooth' at 0x12913213e50>

## **`STEP 3` - Log transform + Scharr edge detection**

- perform a log transformation on the image
- apply Scharr edge detection filter
- combine log image and Scharr edge filtered intensity image

In [ ]:
# log scale the image, apply the scharr edge detection filter to logged image, add the two images together
composite_cellmask = non_linear_cellmask_transform(structure_img_smooth)

# ***CORE-PROCESSING prototype - masks_A***

## **`STEP 4` - Global + local thresholding (AICSSeg – MO) from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package**

- apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)

In [ ]:
# threshold the composite image after log/edge detection using the MO filter function from aicssegmentation - this applies a global threshold, then a local threshold to produce a semantic segmentation
thresh_method = 'ave'
cutoff_size =  50
thresh_adj = 0.2

bw = masked_object_thresh(composite_cellmask, 
                          global_method=thresh_method, 
                          cutoff_size=cutoff_size, 
                          local_adjust=thresh_adj)

# ***POST-PROCESSING prototype - masks_A***

## **`STEP 5` - Remove small holes and objects**

- fill holes (hole size = user input)
- remove small objects (object size = user input)
- filter method = (method = user input)

> ###### 📝 **the size parameters are by convention defined as one dimensional "width", so the inputs to the functions are _squared_ i.e. raised to the power of 2: `**2` for 2D analysis. For volumetric (3D) analysis this would be _cubed_: `**3`.**

In [ ]:
hole_min_width = 0
hole_max_width = 30

small_object_width = 10

fill_filter_method = "3D"

cleaned_img2 = fill_and_filter_linear_size(bw, 
                                           hole_min=hole_min_width, 
                                           hole_max=hole_max_width, 
                                           min_size= small_object_width,
                                           method=fill_filter_method)

###### Labels a boolean version of the cytoplasm mask to a uint16 array

In [ ]:
cytoplasm_mask = label_bool_as_uint16(cleaned_img2)

Define `_infer_cytoplasm_fromcomposite` function

In [ ]:
##########################
# infer_cytoplasm_fromcomposite
# alternative workflow "a"
##########################
def _infer_cytoplasm_fromcomposite(in_img: np.ndarray,
                                  weights: list[int],
                                  median_sz: int,
                                  gauss_sig: float,
                                  mo_method: str,
                                  mo_adjust: float,
                                  mo_cutoff_size: int,
                                  min_hole_w: int,
                                  max_hole_w: int,
                                  small_obj_w: int,
                                  fill_filter_method: str
                                  ) -> np.ndarray:
    """
    Procedure to infer cellmask from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    weights:
        a list of int that corresond to the weights for each channel in the composite; use 0 if a channel should not be included in the composite image
    nuclei_labels: 
        a 3d image containing the inferred nuclei labels
    median_sz: 
        width of median filter for _cellmask_ signal
    gauss_sig: 
        sigma for gaussian smoothing of _cellmask_ signal
    mo_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    mo_adjust: 
        Masked Object threshold `local_adjust`
    mo_cutoff_size: 
        Masked Object threshold `size_min`
    max_hole_w: 
        hole filling cutoff for cellmask signal post-processing
    small_obj_w: 
        minimu object size cutoff for cellmask signal post-processing
    watershed_method:
        determines if the watershed should be run 'sice-by-slice' or in '3D' 

    Returns
    -------------
    cellmask_mask:
        a logical/labels object defining boundaries of cellmask

    """
    ###################
    # EXTRACT
    ###################
    struct_img = weighted_aggregate(in_img, *weights)

    ###################
    # PRE_PROCESSING
    ###################                         
    struct_img = scale_and_smooth(struct_img,
                                   median_size = median_sz, 
                                   gauss_sigma = gauss_sig)
    

    struct_img_non_lin = non_linear_cellmask_transform(struct_img)

    ###################
    # CORE_PROCESSING
    ###################
    struct_obj = masked_object_thresh(struct_img_non_lin, 
                                      global_method=mo_method, 
                                      cutoff_size=mo_cutoff_size, 
                                      local_adjust=mo_adjust)               

    ###################
    # POST_PROCESSING
    ###################
    struct_obj = fill_and_filter_linear_size(struct_obj, 
                                             hole_min=min_hole_w, 
                                             hole_max=max_hole_w, 
                                             min_size= small_obj_w,
                                             method=fill_filter_method)

    ###################
    # POST- POST_PROCESSING
    ###################
    cellmask_out = label_bool_as_uint16(struct_obj)

    return cellmask_out

Run `_infer_cytoplasm_fromcomposite` function

In [ ]:
###################
# PARAMETERS
###################   
weights = [0, 4, 1, 1, 2, 2]
median_sz = 0
gauss_sig = 0
mo_method = "ave"
mo_adjust = 0.2
mo_cutoff_size = 50
hole_min_width = 0
hole_max_width = 30
small_obj_w = 10
fill_filter_method = '3D'

_CY_object = _infer_cytoplasm_fromcomposite(img_data,
                                            weights,
                                            median_sz,
                                            gauss_sig,
                                            mo_method,
                                            mo_adjust,
                                            mo_cutoff_size,
                                            hole_min_width,
                                            hole_max_width,
                                            small_obj_w,
                                            fill_filter_method) 

_CY_object.dtype

In [ ]:
np.all(cytoplasm_mask == _CY_object)

# ***POST-POST-PROCESSING prototype - masks_A***

## **`STEP 6` - Create nucleus mask from cytoplasm mask**

- binary dilation
- fill nucleus (hole size = user input)
- binary erosion

In [ ]:
cytoplasm_dilated = binary_dilation(cytoplasm_mask)

hole_min_width = 0
hole_max_width = 500
small_obj_width = 0
fill_filter_method = "3D"
cytoplasm_filled = fill_and_filter_linear_size(cytoplasm_dilated, hole_min=hole_min_width, hole_max=hole_max_width, min_size=small_obj_width, method=fill_filter_method)

cytoplasm_eroded = binary_erosion(cytoplasm_filled)

- logical **XOR** of the cytoplasm and the filled in cytoplasm resulting in the nucleus and any artifacts from dilation/erosion

In [ ]:
nuclei_xor = np.logical_xor(cytoplasm_mask, cytoplasm_eroded)

- remove small objects (object size = user input)

In [ ]:
nuc_small_object_width = 20

nuc_cleaned_img = fill_and_filter_linear_size(nuclei_xor, hole_min=0, hole_max=0, min_size=nuc_small_object_width, method="3D")

###### Labels a boolean version of the nuclei labels to a uint16 array

In [ ]:
###################
# LABELING
###################
# create instance segmentation based on connectivity
nuclei_labels = label_bool_as_uint16(nuc_cleaned_img)

In [ ]:
nuclei_labels.dtype

Define `_infer_nuclei_fromcytoplasm` function

In [ ]:
##########################
#  _infer_nuclei_fromcytoplasm
##########################
def _infer_nuclei_fromcytoplasm(cytoplasm_mask: np.ndarray, 
                                nuc_min_hole_w: int,
                                nuc_max_hole_w: int,
                                nuc_fill_method: str,
                                small_obj_w: int,
                                fill_filter_method: str
                                ) -> np.ndarray:
    """
    Procedure to infer nuclei from linear unmixed input.

    Parameters
    ------------
    cytoplasm_mask: np.ndarray
        a 3d image of the cytoplasm segmentation
    max_hole_w: int
        hole filling cutoff to fill the nuclei
    small_obj_w: int
        object size cutoff to remove artifacts from dilation/erosion steps
    fill_filter_method: str
        to filter artifacts in "3D" or "slice-by-slice"

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """

    ###################
    # PRE_PROCESSING
    ###################                
    cytoplasm_dilated = binary_dilation(cytoplasm_mask)

    cytoplasm_filled = fill_and_filter_linear_size(cytoplasm_dilated, 
                                                   hole_min=nuc_min_hole_w, 
                                                   hole_max=nuc_max_hole_w, 
                                                   min_size=0, 
                                                   method=nuc_fill_method)

    cytoplasm_eroded = binary_erosion(cytoplasm_filled)

    ###################
    # CORE_PROCESSING
    ###################
    nuclei_xor = np.logical_xor(cytoplasm_mask, cytoplasm_eroded)

    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = fill_and_filter_linear_size(nuclei_xor, 
                                                hole_min=0, 
                                                hole_max=0, 
                                                min_size=small_obj_w,
                                                method=fill_filter_method)

    nuclei_labels = label_uint16(nuclei_object)

    return nuclei_labels

Run `_infer_nuclei_fromcytoplasm` function

In [ ]:
nuc_min_hole_w = 0
nuc_max_hole_w = 500
nuc_fill_method = "3D"
small_obj_w = 20
fill_filter_method = "3D"

_NU_object = _infer_nuclei_fromcytoplasm(cytoplasm_mask,
                                nuc_min_hole_w,
                                nuc_max_hole_w,
                                nuc_fill_method,
                                small_obj_w,
                                fill_filter_method)

_NU_object.dtype

## **`STEP 7` - Create cellmask from cytoplasm and nucleus mask**

- logical **OR** of the nucleus and cytoplasm

In [ ]:
cell = np.logical_or(_NU_object, _CY_object)

- fill small holes (hole size = user input)
- remove small objects (object size = user input)

In [ ]:
min_hole_width = 0
max_hole_width = 30
small_obj_width = 0
fill_method = "3D"

cleaned_img = fill_and_filter_linear_size(cell, 
                                          hole_min=min_hole_width, 
                                          hole_max=max_hole_width, 
                                          min_size=small_obj_width, 
                                          method=fill_method)

###### Labels a boolean version of the cell mask to a uint16 array

In [ ]:
# create instance segmentation based on connectivity
cell_mask = label_bool_as_uint16(cleaned_img)

In [ ]:
cell_mask.dtype

# ***EXPORT prototype - masks_A***

## **`STEP 8` - Stack masks**

- Stack masks in order of nucleus, cellmask and cytoplasm mask

In [ ]:
stack = stack_masks(nuc_mask=_NU_object, cellmask=cell_mask, cyto_mask=_CY_object)

In [ ]:
out_file_n = export_inferred_organelle(stack, "masks_A", meta_dict, out_data_path)

## **Visualize `nucleus`, `cellmask` and `cytoplasm`**

In [ ]:
viewer_masks_A = napari.Viewer(title = "masks_A",
                           ndisplay=3)
viewer_masks_A.grid.enabled = True

In [ ]:
viewer_masks_A.add_image(stack[2].astype(bool),
                      scale = scale,
                      name = 'Cytoplasm')

viewer_masks_A.add_image(stack[1].astype(bool),
                      scale = scale,
                      name = 'Cellmask')

viewer_masks_A.add_image(stack[0].astype(bool),
                      scale = scale,
                      name = 'Nucleus')

viewer_masks_A.reset_view()

nbscreenshot(viewer_masks_A,
             canvas_only = True)

-------------
### NEXT: INFER LYSOSOME

proceed to [1.2_infer_lysosome.ipynb](./1.2_infer_lysosome.ipynb)